This is the code for the parameter-free threshold selection algorithm for extremal index estimation proposed by Markovich and Rodionov (2022). The result of the algorithm is an extremal index estimate. Please, follow the instructions below.

In [44]:
#Here we import some necessary libraries. Please, run this cell.
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [1]:
#Please, upload a csv-file with your data here. This csv should contain an one-dimensional array of real numbers.
from google.colab import files
uploaded = files.upload()

In [40]:
#Please, write the name of the uploaded file in quotes below. Example: myfile.csv
data = pd.read_csv("", sep = ";", index_col=False)

In [5]:
#In this cell, the intervals and K-gaps estimators of the extremal index are implemented. Also, we implement here the algorithm for calulating
#inter-exceedance times, see Section 2, before (4), in Markovich and Rodionov (2022). Please, run this cell.

def inter_exceedance_times(sample, level):

        sample_len = len(sample)
        S_previous = 0
        T = []
        flag = 0

        for i in np.arange(sample_len):
            if flag == 0:
                if sample[i] > level:
                    flag = 1
                    S_previous = i
            else:
                if sample[i] > level:
                    T.append(i - S_previous)
                    S_previous = i

        return np.array(T)

def intervals_estimator(T): 

    #Here, the intervals estimator is implemented, see (4) in Markovich and Rodionov (2022), T is an array of inter-exceedance times
    if np.max(T) <= 2:
            theta = min(1, (2 * (np.mean(T)) ** 2) / (np.mean(T ** 2)))
    else:
            theta = min(1, (2 * (np.mean(T - 1)) ** 2) / (np.mean(np.multiply((T - 1), (T - 2)))))

    return theta

def Kgaps_estimator(Y, N_C, L):

    #Here, the K-gaps estimator is implemented, see (6) in Markovich and Rodionov (2022), Y are the normalized K-gaps
    a = L - N_C
    b = 2 * N_C
    c = np.sum(Y)
    theta = 0.5 * ((a + b) / c + 1 - (((a + b) / c + 1) ** 2 - 4 * b / c) ** 0.5)

    return theta

In [28]:
#Here, we implement the Algorithm 1 for the intervals and K-gaps estimators. Please, run this cell.
def discrepancy_intervals(data, val_num, quantiles_levels):

    q_len = len(quantiles_levels)
    #delta_2 is used in the third step of the Algorithm 1, see formula (16).
    delta_2 = 1.49

    thetas = np.zeros(q_len)
    levels_opt = np.zeros(q_len)
    thetas_opt = np.zeros(q_len)

    for u_c in np.arange(q_len):
        #Implementation of the first step of the Algorithm 1
        level = np.percentile(data, quantiles_levels[u_c])
        T = inter_exceedance_times(data, level)
        L = len(T)
        if L == 0:
          break
        Y = (L + 1) * T / len(data)
        Z = np.sort(Y)

        #Here, we calculate a pilot intervals estimate and k, implementing the second step of the Algorithm 1
        thetas[u_c] = intervals_estimator(T)
        k = int(np.floor(thetas[u_c] * L)) 
        #Other options for k are [min(sqrt(L),theta(u_c)*L)] and [(log(L))^2], but we recommend to use the option in the previous line

        if thetas[u_c] == 1:
            k = L 
        if L > 1:
            Z_0 = Z[L - k - 1]
        else:
            Z_0 = 0

        #Here, implementing step 3 of the Algorithm, we calculate the omega-squared statistic and compare its value with delta_2
        if k > 0:
            omega_squared = 1 / (12 * k)
            for j in np.arange(k):
                omega_squared += (1 - np.exp(- thetas[u_c] * (Z[L - k + j] - Z_0)) - (j + 0.5) / k) ** 2

            if L < 40:
                omega_squared = (omega_squared - 0.4 / L + 0.6 / L ** 2) * (1 + 1 / L)

            if omega_squared <= delta_2:
                levels_opt[u_c] = u_c
                thetas_opt[u_c] = thetas[u_c]

    #Finally, we implement the fourth step of the Algorithm 1
    if np.sum(thetas_opt != 0) == 0:
        return 0
    else:
        b = thetas_opt[thetas_opt != 0]
        c = levels_opt[levels_opt != 0]
        if val_num == 1:
            return np.mean(b)
        elif val_num == 2:       
            c_min = np.min(c)
            return thetas_opt[int(c_min)]
        elif val_num == 3:
            c_max = np.max(c)
            return thetas_opt[int(c_max)]


def discrepancy_Kgaps(data, val_num, quantiles_levels):
    q_len = len(quantiles_levels)
    #delta_2 is used in the third step of the Algorithm 1, see formula (16).
    delta_2 = 1.49
    # It is a standard value of maximal K for K-gaps estimator, see Suveges and Davison (2010) or Remark 4 in Markovich and Rodionov (2022) 
    K_max = 20

    thetas = np.zeros(q_len)
    thetas_Kgaps = np.zeros((K_max, q_len))
    levels_opt = np.zeros((K_max, q_len))
    K_opt = np.zeros((K_max, q_len))
    thetas_opt = np.zeros((K_max, q_len))

    for u_c in np.arange(q_len):
        level = np.percentile(data, quantiles_levels[u_c])
        flag = 0
        
        #Implementation of the first step of the Algorithm 1, see also Remark 4 in Markovich and Rodionov (2022)
        level = np.percentile(data, quantiles_levels[u_c])
        T = inter_exceedance_times(data, level)
        n = len(T) + 1
        if n == 1:
          break 
        N = len(data) 
        #Here, we calculate a pilot intervals estimate, implementing the second step of the Algorithm 1; k is calculated below
        thetas[u_c] = intervals_estimator(T)

        for K in np.arange(1, K_max + 1):
            S = np.where(T > K, T - K, 0)
            Y = n * S / N
            Z = np.sort(Y)
            L = len(Y)
            N_C = len(Y[Y != 0])

            k = int(np.floor(thetas[u_c] * L))
            if k >= L:
                k = L
            if L > 1:
                Z_0 = Z[L - k - 1]
            else:
                Z_0 = 0

            #Here, implementing step 3 of the Algorithm, we calculate the omega-squared statistic and compare its value with delta_2
            #Note that we use here K-gaps estimator instead of intervals estimator
            if k > 0:
                thetas_Kgaps[K - 1, u_c] = Kgaps_estimator(Y, N_C, L)
                omega_squared = 1 / (12 * k)
                for j in np.arange(k):
                    omega_squared += (1 - np.exp(- thetas_Kgaps[K - 1, u_c] * (Z[L - k + j] - Z_0)) - (j + 0.5) / k) ** 2

                if L < 40:
                    omega_squared = (omega_squared - 0.4 / L + 0.6 / L ** 2) * (1 + 1 / L)

                if omega_squared <= delta_2:
                    levels_opt[K - 1, u_c] = u_c
                    K_opt[K - 1, u_c] = K
                    thetas_opt[K - 1, u_c] = thetas_Kgaps[K - 1, u_c]
                else:
                    levels_opt[K - 1, u_c] = np.NAN
                    K_opt[K - 1, u_c] = np.NAN
    #Finally, we implement the fourth step of the Algorithm 1 with necessary changes
    if np.sum(thetas_opt != 0) == 0:
        return 0
    else:
        if val_num == 1:
            b = thetas_opt[thetas_opt != 0]
            return np.nanmean(b)
        elif val_num == 2:
            c_min = np.nanmin(levels_opt)
            return np.nanmean(thetas_opt[:, int(c_min)])
        elif val_num == 3:
            c_max = np.nanmax(levels_opt)
            return np.nanmean(thetas_opt[:, int(c_max)])

In [42]:
#Here, you can choose between two options: run the Algorithm 1 by Markovich and Rodionov (2022) with either the intervals or the K-gaps estimator.
#If you choose the intervals estimator, please set est_num = 1, if K-gaps, then set est_num = 2. We recommend to use the second option.
est_num = 2

#At the end of Algorithm 1, there are three options for the final value of the estimator, see formula (15) in Markovich and Rodionov (2022).
#Please, specify, which one you want to use: set val_num = 1, if you select theta_1 (average); val_num = 2, if you select theta_2 (estimate with 
#minimal threshold level), val_num = 3, if you select theta_3 (estimate with maximal threshold level). We recommend to use the first option.
val_num = 1

#Here we set the quantile levels which are defined in the first step of the Algorithm 1. You can set another values, if it is necessary.
quantiles_levels = np.array([90, 90.5, 91, 91.5, 92, 92.5, 93, 93.5, 94, 94.5, 95, 95.5, 96, 96.5, 97, 97.5, 98, 98.5, 99, 99.5])

In [45]:
# Please, run this cell to get the extremal index estimate
if est_num == 1:
    print("The extremal index is", discrepancy_intervals(data, val_num, quantiles_levels))
elif est_num == 2:
    print("The extremal index is", discrepancy_Kgaps(data, val_num, quantiles_levels))


The extremal index is 0.9028651582294737
